# imports

In [ ]:
import json
import os
from tqdm import tqdm
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from sentencepiece import SentencePieceProcessor

# paths

In [4]:
data = './data'

In [5]:
tokenizer_path = './tokenizer.model'
# this is the same tokenizer as found in llama2

In [25]:
out_data = './instruct_data_same_length/'
os.makedirs(out_data, exist_ok=True)

# preprocessing functions

In [7]:
def replace_words(
    prompt, 
    words_list
):
    for words_to_remove, replacement in words_list:
        prompt = prompt.replace(words_to_remove, replacement)
    return prompt
        

In [8]:
def preprocess_prompt(prompt, suffix=' Possible story:'):
    words_list = [
    ("Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand.", "Write a story."),
    (" Remember to only use simple words!", ""),
    ("\n\nPossible story:", ""),
    ("try to at some point use", "try to use")
    ]
    prompt = replace_words(prompt, words_list)
    return prompt + suffix

# tokenize

In [11]:
tokenizer = SentencePieceProcessor(model_file=tokenizer_path)

In [12]:
print(tokenizer.vocab_size())
print(tokenizer.bos_id(), tokenizer.eos_id())

32000
1 2


In [13]:
np.array([-100]*15, dtype=np.uint16)

array([65436, 65436, 65436, 65436, 65436, 65436, 65436, 65436, 65436,
       65436, 65436, 65436, 65436, 65436, 65436], dtype=uint16)

In [51]:
def tokenize_chunk(chunk_path, out_folder, max_seq_len=350, pad_token=-100):
    
    print(chunk_path)
    with open(chunk_path, 'r') as f:
        chunk = json.load(f)
    all_tokens = []
    all_labels = []
    for sample in tqdm(chunk):
        story = sample['story'].strip()
        prompt = preprocess_prompt(sample['instruction']['prompt:'].strip())
        tokenized_prompt = tokenizer.encode(prompt)
        
        prompt_and_story = tokenized_prompt + [tokenizer.bos_id()] + tokenizer.encode(story) + [tokenizer.eos_id()]
        label = [pad_token]*len(tokenized_prompt) + [tokenizer.bos_id()] + tokenizer.encode(story) + [tokenizer.eos_id()]

        if len(prompt_and_story) <= max_seq_len:
            prompt_and_story += [pad_token] * (max_seq_len - len(prompt_and_story))
            label += [pad_token] * (max_seq_len - len(label))
            assert len(prompt_and_story) == len(label) == max_seq_len
            all_tokens.extend(prompt_and_story)
            all_labels.extend(label)
    #print(len(all_tokens))
    all_tokens = np.array(all_tokens, dtype=np.uint16)
    all_labels = np.array(all_labels, dtype=np.uint16)
   # print(all_tokens.shape)
    
    all_tokens_filename = chunk_path.split('/')[-1].replace('.json', '.bin')
    with open(os.path.join(out_folder, all_tokens_filename), "wb") as f:
        f.write(all_tokens.tobytes())
        print(os.path.join(out_folder, all_tokens_filename))
    
    all_labels_filename = all_tokens_filename.replace('data', 'labels')
    with open(os.path.join(out_folder, all_labels_filename), "wb") as f:
        f.write(all_labels.tobytes())
        
    
def tokenize_all_chunks(data, out_folder, max_workers=5):
    tokenize_chunk_args = [(os.path.join(data, fn), out_folder) for fn in os.listdir(data) if fn.endswith('.json')]
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        executor.map(tokenize_chunk, *zip(*tokenize_chunk_args))
        
        

In [52]:
tokenize_all_chunks(data=data, out_folder=out_data)

./data/data42.json./data/data17.json./data/data30.json./data/data11.json./data/data14.json






 93%|███████████████████████████████████████████████████████████████████▊     | 92813/100000 [01:39<00:07, 1006.25it/s]

./instruct_data_same_length/data11.bin


 93%|████████████████████████████████████████████████████████████████████▊     | 93019/100000 [01:40<00:07, 978.35it/s]

./data/data26.json


...

./instruct_data_same_length/data44.bin


 80%|██████████████████████████████████████████████████████████▍              | 80070/100000 [01:16<00:18, 1067.08it/s]

./instruct_data_same_length/data41.bin


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [01:33<00:00, 1068.14it/s]


./instruct_data_same_length/data16.bin


In [53]:
with open(out_data + 'data01.bin', 'rb') as f:
    x = f.read()
x = np.frombuffer(x, dtype=np.uint16)

In [54]:
len(x)/350

94053.0

In [55]:
with open(out_data + 'labels01.bin', 'rb') as f:
    y = f.read()
y = np.frombuffer(y, dtype=np.uint16)

In [56]:
len(y)/350

94053.0